# Cargar datos y librerias

In [1]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import shift
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from xgboost import XGBRegressor

In [2]:
from warnings import filterwarnings
filterwarnings("ignore")

In [3]:
import chart_studio.plotly as ply
import cufflinks as cf
cf.go_offline()

In [4]:
def cargar():
    # variables climaticas
    consumofeb=pd.read_excel(r'./Consumo_feb.xlsx').set_index("fecha").loc["2007-01-01":] #desde 2007 hasta 31 de marzo de 2020
    consumofeb = consumofeb.drop(["prom", 'Lluvia', 'Velocidad_viento'], axis = 1)
    #variables de consumo MWh
    consumo=pd.read_excel('./Data1.xlsx').set_index("fecha").loc["2007-01-01":"2020-03-30"] 
    return consumo, consumofeb

consumo, consumofeb = cargar()

# pred clima

In [5]:
def difs():
    # Inicializar posicion de datos variables difusas

    inv1 = [310.5-365.25, 8.6529, 138.941176470588]
    pr = [8.6529, 138.941176470588, 221.9375]
    vr = [138.941176470588, 221.9375, 310.5]
    ot = [221.9375, 310.5, 365.25+8.6529]
    ot2 = [221.9375-365.25, 310.5-365.25, 8.6529]
    inv2 = [310.5, 365.25+8.6529, 365.25+138.941176470588]

    height = {'Invierno1': inv1,'Primavera': pr,'verano': vr,'Otoño': ot,'Otoño2': ot2,'Invierno2': inv2,} 
    return height

# para crear variables difusas
def triangular():
    
    # Se crea x
    ma = pd.DataFrame();
    x = consumofeb.index.dayofyear.to_list()
    # Se crean variables a, b y c
    height = difs()
    for hei in height:
        ba = pd.DataFrame(x, columns=["x"])
        ba["a"] = height[hei][0]
        ba["b"] = height[hei][1]
        ba["c"] = height[hei][2]
        # Variable difusa
        ca = pd.DataFrame()
        ca["bo"] = (ba.x-ba.a)/(ba.b-ba.a)
        ca["ba"] = (ba.c-ba.x)/(ba.c-ba.b)
        ca = ca.min(axis = 1)
        ca[ca<0]=0
        ma = pd.concat([ma, ca], axis = 1) # Se concatenan

    ma.index = consumofeb.index
    ma.columns = height.keys()
    return ma

In [6]:
def kronecker(data1:'Dataframe 1',data2:'Dataframe 2'):
    Combinacion = pd.DataFrame(); d1 = pd.DataFrame()
    for i in data2.columns:
        d1 = data1.multiply(data2[f"{i}"] , axis="index")
        d1.columns = [f'{i}_{j}' for j in data1.columns]
        Combinacion = pd.concat([Combinacion, d1], axis = 1)
    return Combinacion

In [7]:
def pba(j, n):
    # armonicos
    f, asd = signal.periodogram(consumofeb[j], 1)

    picos = pd.DataFrame(asd, 1/(f), columns=["potencia"])
    picos = picos.sort_values(by="potencia",ascending=False).reset_index().head(12)
    picos.columns = ["periodo", "potencia"]
    #fourier
    sencos = pd.DataFrame(index = consumofeb.index)
    t = np.arange(1,len(consumofeb)+1)
    sencos["t"]=t
    for i  in  picos.periodo:
            sencos[f"{i:.2f}_sen"] = np.abs(np.sin(((2*np.pi)/(i))*t))
            sencos[f"{i:.2f}_cos"] = np.abs(np.cos(((2*np.pi)/(i))*t))
    sencos['ones']=1
    sencos['sen1']=np.abs(np.sin(((2*np.pi)/(365.25))*t))
    sencos['cos1']=np.abs(np.cos(((2*np.pi)/(365.25))*t))
    
    sencos=kronecker(sencos, triangular())
    #regresion
    X_train, X_test, y_train, y_test = train_test_split(sencos, consumofeb[j], test_size=n, shuffle = False)
    
    reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
    y_predict=reg.predict(X_test)
    
    nov=pd.DataFrame(np.concatenate([reg.predict(X_train),y_predict]),index=consumofeb.index,columns=[f"{j}"])
    return nov

def modelosclima(n):
    ca = pd.DataFrame(index = consumo.index)
    for i in ["mín", "max", "Nubosidad"]:
        ca = pd.concat([ca, pba(i, n)], axis = 1)
    return ca

modclima = modelosclima(8)

# funciones

In [8]:
def Mape(y,y_pred):
    return np.mean(np.abs((y-y_pred)/y))*100

In [9]:
def festivos():
    festivos=pd.read_excel('./Festivos.xlsx')
    festivos2=pd.read_excel('./Festivos2.xlsx')
#     festivos3=pd.read_excel('./Festivos3.xlsx')
    
    Dum = pd.DataFrame(index = consumo.index)

    for col in festivos.columns:
        Dum[f"{col}"] = consumo.index.isin(festivos[f"{col}"])
        Dum[col] = Dum[col].replace([False, True],[0, 1])
        clean = Dum[col].values
        Dum[col] = Dum[col]+shift(clean, 1, cval=0)*.4
        Dum[col] = Dum[col]+shift(clean, -1, cval=0)*.4
        Dum[col] = Dum[col]+shift(clean, 2, cval=0)*.1
        Dum[col] = Dum[col]+shift(clean, -2, cval=0)*.1
        Dum[col] = Dum[col]/2
        
#     for col in festivos3.columns:
#         Dum[f"{col}"] = consumo.index.isin(festivos3[f"{col}"])
#         Dum[col] = Dum[col].replace([False, True],[0, 1])
#         clean = Dum[col].values
# #         Dum[col] = Dum[col]+shift(clean, -1, cval=0)*2
#         Dum[col] = Dum[col]+shift(clean, 1, cval=0)/3
#         Dum[col] = Dum[col]+shift(clean, -1, cval=0)
#         Dum[col] = Dum[col]
        
        
# #     Dum["ones"] = 1
    tm = np.arange(1,len(consumo.index)+1)  
    Dum["t"] = tm/np.max(tm)
    
    
    for col in festivos2.columns:
        Dum[col] = consumo.index.isin(festivos2[col])
    Dum = Dum.replace([False, True],[0, 1])
    
    return Dum

In [10]:
def dummies(n):   
    # variables eureka y clima
    consumo2 = pd.DataFrame(index=consumo.index)
    consumo2["num"] = np.arange(1,len(consumo.index)+1)
    consumo2["day"] = consumo.index.weekday
    consumo2["eureka1"] = consumo2.day**3
    consumo2["eureka2"] = modclima["max"]*consumo2.num
    consumo2["month"] = consumo.index.month
    X = consumo2.join(modclima)
    # normalizacion eureka-clima
    X_norm = X/X.max()
    
    # variables gplearn
    X_norm["gplearn1"]=np.cos(X_norm.day)*X_norm.num
    X_norm["gplearn2"]=X_norm.num*X_norm["mín"]
    X_norm["gplearn3"]=3.319**X_norm["max"]*np.cos(X_norm.day)
    X_norm["gplearn8"]=3.319**X_norm["max"]*X_norm.num 
    X_norm["gplearn4"]=np.cos(X_norm.day)*np.sin(X_norm.day)
    X_norm["gplearn6"]=X_norm["max"]**2*X_norm["mín"]**2
    X_norm["gplearn7"]=1/np.cos(X_norm.day)
    X_norm["gplearn10"]=X_norm["max"]*X_norm["mín"]**3*np.cos(X_norm.month)**2
    
    X_norm=X_norm.drop(['mín', 'max', 'Nubosidad'],axis=1)
    # variables dummies
    X_norm = pd.get_dummies(X_norm, columns=["month", "day"], prefix=["month", "day"], drop_first=True)

    # variables festivos y difusa de clima
    X_norm = X_norm.join(festivos())
    X_norm = X_norm.join(triangular())
    
    # variables entre y fin
    X_norm["fin"]=consumo.index.weekday
    X_norm["entre"]=consumo.index.weekday
    X_norm["fin"]=X_norm["fin"].replace([0,1,2,3,4,5,6],[.5,0,0,0,.5,1,1])
    X_norm["entre"]=X_norm["entre"].replace([0,1,2,3,4,5,6],[.5,1,1,1,.5,0,0])
    return X_norm

In [11]:
def separar(n, X, y):
    # separar X y Y
    X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                  test_size = n, shuffle = False)
    return (X_train, X_test), (y_train, y_test)

In [12]:
# exog sin ones, porque son constantes

# Primer rgresion

In [13]:
def modreg(regresor):
    while True:
        if regresor == "Lr":
            cok = LinearRegression(normalize = True, n_jobs =-1)
            return cok
        elif regresor == "Lasso":
            cok = Lasso(alpha=0, fit_intercept = False, max_iter = 7000, precompute = True, tol = 0.0)
            return cok
        elif regresor == "Ridge":
            cok = Ridge()
            return cok
        elif regresor == "xgb":
            cok = XGBRegressor(n_jobs = -1, reg_alpha=100,
                               objective = "reg:squarederror", max_depth  = 3, n_estimatiors = 1000) #tree_method= "gpu_hist", nthread=-1,
            return cok
        else: 
            raise NameError(f"{regresor} was not found") #reg_alpha=100,
            break

## regresion gen_for

In [14]:
def reg(n, pre_forc, nom_reg1): # n = 7 , pre_for = "general" o "forecasting"
    # cargar X y Y
    y = consumo; X = dummies(n)
    # Se obtienen los datos separados
    Xo, yo = separar(n, X, y)  
    
    # Decision "general" "forecasting"
    while True:
        if pre_forc == "general": 
            pre_forc = 0
            break
        elif pre_forc == "forecasting": 
            pre_forc = 1
            break
        else: 
            raise NameError(f"expected 'general' or 'forecasting', got '{pre_forc}' instead")
            break
        
    reg1 = modreg(nom_reg1).fit(Xo[0], yo[0])
    y_predict=reg1.predict(Xo[pre_forc])
    
    # dataframe real-predict- resta
    comp = pd.DataFrame(y_predict, columns = ["predict"],index = yo[pre_forc].index)
    comp["real"] = yo[pre_forc]; comp["resta"] = comp.real - comp.predict
    comp["error"] = np.abs((comp.real - comp.predict)/ comp.real)*100
    return comp

In [27]:
bab = reg(365, "forecasting", "Lasso")
bab[["real", "predict"]].iplot(title = bab.error.mean())

In [23]:
for i in ["Lr", "Lasso", "Ridge", "xgb"]:
    print(reg(365, "forecasting", i).error.mean())

2.4272046099782187
2.2845456843210266
2.4823226531263405
2.380340859791674


In [29]:
for i in ["Lr", "Lasso", "Ridge", "xgb"]:
    print(reg(7, "forecasting", i).error.mean())

3.3747274960262663
3.3557265437501593
3.1724345803032783
1.7288199532941835


In [92]:
ba = reg(365, "forecasting", "Lasso")
ba[["real", "predict"]].loc["2020-01-25":"2020-02-10"].iplot(title = ba.error.mean() ,xTitle = "fecha", yTitle = "consumo") #title = ba.error.mean() 

## regresion iterativa

In [120]:
def reg_it(n, it, nom_reg):
    cas = {}; mape = []; y = consumo; X = dummies(n)
    itera = np.flip(np.arange(1, it))
    for i in itera:
        j = i*n

        X_train = X.head(len(y)-j); X_test = X.tail(j).head(n)
        y_train = y.head(len(y)-j); y_test = y.tail(j).head(n)

        regre = modreg(nom_reg).fit(X_train,y_train)

        y_pred = regre.predict(X_test) # tree_method=’gpu_hist’
        y_test["predict"] = y_pred
        y_test["resta"] = y_test.MWh - y_test.predict
        y_test["error"] = (np.abs((y_test.MWh - y_test.predict)/y_test.MWh))*100
#         print(y_test.error.mean())
        mape.append(y_test.error.mean())
        cas[i] = y_test
    print(sum(mape)/(it-1))
    return cas

In [189]:
bos = reg_it(7, 53, "xgb")

2.2081930057925936


In [169]:
# bos[3][["MWh", "predict"]].iplot(title = bos[3].error.mean())

In [25]:
for i in ["Lr", "Lasso", "Ridge", "xgb"]:
    print(i)
    reg_it(7, 53, i)

Lr
2.2774567502054497
Lasso
2.1936002542372877
Ridge
2.3841232844402294
xgb
2.1665289868153748


In [190]:
for i in ["Lr", "Lasso", "Ridge", "xgb"]:
    print(i)
    reg_it(7, 53, i)

Lr
2.3212539954017086
Lasso
2.2357848745110815
Ridge
2.4299718091410027
xgb
2.2081930057925936


# Fourier

## resta

In [36]:
def rego(n, nom_reg1): # n = 7 , pre_for = "general" o "forecasting"
    # cargar X y Y
    y = consumo; X = dummies(n)
    # Se obtienen los datos separados
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = n, shuffle = False)
        
    reg1 = modreg(nom_reg1).fit(X_train, y_train)
    y_predict=reg1.predict(X)
    
    # dataframe real-predict- resta
    comp = pd.DataFrame(y_predict, columns = ["predict"],index = y.index)
    comp["real"] = y; comp["resta"] = comp.real - comp.predict
    comp["error"] = np.abs((comp.real - comp.predict)/ comp.real)*100
    return comp

In [42]:
def regfourier(n, pre_forc, nom_reg1): # n = 7 , pre_for = "general" o "forecasting"
    # cargar X y Y
    y = rego(n, nom_reg1).resta; X = dummies(n)
    train_test_split(X)
    # Se obtienen los datos separados
    Xo, yo = separar(n, X, y)  
    
    # Decision "general" "forecasting"
    while True:
        if pre_forc == "general": 
            pre_forc = 0
            break
        elif pre_forc == "forecasting": 
            pre_forc = 1
            break
        else: 
            raise NameError(f"expected 'general' or 'forecasting', got '{pre_forc}' instead")
            break
        
    reg1 = modreg(nom_reg1).fit(Xo[0], yo[0])
    y_predict=reg1.predict(Xo[pre_forc])
    
    # dataframe real-predict- resta
    comp = pd.DataFrame(y_predict, columns = ["predict"],index = yo[pre_forc].index)
    comp["real"] = yo[pre_forc]; comp["resta"] = comp.real - comp.predict
    comp["error"] = np.abs((comp.real - comp.predict)/ comp.real)*100
    return comp

In [44]:
regfourier(1, "general", "xgb").error.mean()

189.1613660374007

In [46]:
regfourier(1, "general", "xgb")[["real", "predict"]].iplot()

## resta iterativa

In [103]:
def regfourier_it(n, it, nom_reg):
    cas = {}; mape = []; y = rego(n, nom_reg).resta; X = dummies(n)
    itera = np.flip(np.arange(1, it))
    for i in itera:
        j = i*n

        X_train = X.head(len(y)-j); X_test = X.tail(j).head(n)
        y_train = y.head(len(y)-j); y_test = y.tail(j).head(n)

        regre = modreg(nom_reg).fit(X_train,y_train)

        y_predict = regre.predict(X_test) # tree_method=’gpu_hist’
        
        comp = pd.DataFrame(y_predict, columns = ["predict"],index = y_test.index)
        comp["real"] = y_test; comp["resta"] = comp.real - comp.predict
        comp["error"] = np.abs((comp.real - comp.predict)/ comp.real)*100
        mape.append(comp.error.mean())
        cas[i] = comp
    print(sum(mape)/(it-1))
    return cas

## suma

In [106]:
def suma(n, pre_forc, nom_reg1):
    ba = reg(n, pre_forc, nom_reg1)
    be = regfourier(n, pre_forc, nom_reg1)
    bi = pd.DataFrame(ba.predict + be.predict)
    bi["real"] = ba.real
    bi["error"] = np.abs((bi.real - bi.predict)/ bi.real)*100
    return bi

In [149]:
run = suma(7, "forecasting", "xgb")
run[["real", "predict"]].iplot(title = run.error.mean())

## iterativa

In [153]:
def suma_it(n, it, nom_reg):
    
    ba = reg_it(n, it, nom_reg)
    be = regfourier_it(n, it, nom_reg)
    itera = np.flip(np.arange(1, it))
    cas = {}
    for i in itera: 
        bi = pd.DataFrame(ba[i].predict + be[i].predict)
        bi["real"] = ba[i].MWh
        bi["error"] = np.abs((bi.real - bi.predict)/ bi.real)*100
        cas[i] = bi
    return cas

In [154]:
ab = suma_it(7, 53, "xgb")

2.2081930057925936
374.28092783890065


In [180]:
do = 0
for i in range(1, 53):
    do += ab[i].error.mean()

In [179]:
num = 9
print(bos[num].error.mean())
print(ab[num].error.mean())

2.268652488437021
1.3810076670343396


In [183]:
do/52

2.1729022069063997

In [ ]:
bos[3][["MWh", "predict"]].iplot(title = bos[3].error.mean())

# varios a la vez

In [ ]:
def aro(n):
    X = dummies(n)
    X_train, X_test, y_train, y_test = train_test_split(X, consumo, test_size = n, shuffle = False)
    model = ARIMA(y_train.values, exog = X_train,  order=(7,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast(steps = n, exog = X_test); predo = output[0]
    y_test["predict"] = predo
    y_test["resta"] = y_test.MWh - y_test.predict
    y_test["error"] = np.abs((y_test.MWh - y_test.predict)/y_test.MWh)*100
    return y_test

In [ ]:
%time ab = aro(7)
ab[["MWh", "predict"]].iplot(title = ab.error.mean())

# CASA

In [ ]:
cas = pd.read_excel('./Festivos2.xlsx').Benito

In [ ]:
from datetime import timedelta

In [ ]:
bob = []
for i in cas.index: 
    bob.append(cas.loc[i] - datetime.timedelta(days=1))
    bob.append(cas.iloc[i])

In [ ]:
pd.DataFrame(bob).set_index(0).to_excel("bob.xlsx")

In [175]:
cas.loc[0] + datetime.timedelta(days=1)

Timestamp('2004-03-22 00:00:00')

In [196]:
cas

0    2004-03-20
1    2004-03-21
2    2005-03-20
3    2005-03-21
4    2006-03-20
5    2006-03-21
6    2007-03-18
7    2007-03-19
8    2008-03-16
9    2008-03-17
10   2009-03-15
11   2009-03-16
12   2010-03-14
13   2010-03-15
14   2011-03-20
15   2011-03-21
16   2012-03-18
17   2012-03-19
18   2013-03-17
19   2013-03-18
20   2014-03-16
21   2014-03-17
22   2015-03-15
23   2015-03-16
24   2016-03-20
25   2016-03-21
26   2017-03-19
27   2017-03-20
28   2018-03-18
29   2018-03-19
30   2019-03-17
31   2019-03-18
32   2020-03-15
33   2020-03-16
Name: Benito, dtype: datetime64[ns]